In [1]:
import math
import os
import re
import itertools

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

%matplotlib inline

In [2]:
def read(file_name):
    data_dir = '/Users/keiji/work/kaggle/sales1c/'
    pickle = data_dir + file_name + '.pickle'
    if os.path.exists(pickle):
        return pd.read_pickle(pickle)
    df = pd.read_csv(data_dir + file_name)
    df.to_pickle(pickle)
    return df

df_icats = read('item_categories.csv')
df_items = read('items.csv')
df_shops = read('shops.csv')
df_test = read('test.csv.gz')
df_sales = read('sales_train.csv.gz')

In [3]:
# Drop outliers.
# I'm not sure if this really matters.
df_sales = df_sales[df_sales.item_price < 60000]
df_sales = df_sales[df_sales.item_cnt_day < 700]

In [4]:
df_sales['item_sales'] = df_sales.item_price * df_sales.item_cnt_day

In [5]:
unique_shop_id = df_sales[['date_block_num', 'shop_id']].drop_duplicates().reset_index(drop=True)
unique_item_id = df_sales[['date_block_num', 'item_id']].drop_duplicates().reset_index(drop=True)
df_train = (
    pd.DataFrame({'date_block_num': np.arange(34)})
    .merge(unique_shop_id, how='left', on='date_block_num')
    .merge(unique_item_id, how='left', on='date_block_num')
    .merge(df_items[['item_id', 'item_category_id']], how='left', on='item_id')
    .sort_values(by=['date_block_num', 'shop_id', 'item_id'])
    .reset_index(drop=True)
)

df_agg = (
    df_sales.groupby(['date_block_num', 'shop_id', 'item_id'])
    .agg({
        'item_cnt_day': 'sum',
        'item_price': 'median',
        'item_sales': 'sum',
    })
    .reset_index()
    .rename(columns={
        'item_cnt_day': 'item_cnt_month',
        'item_price': 'median_price',
        'item_sales': 'item_sales_month',
    })
)
df_train = df_train.merge(df_agg, how='left', on=['date_block_num', 'shop_id', 'item_id'])
df_train.item_cnt_month.fillna(0.0, inplace=True)
df_train.item_sales_month.fillna(0.0, inplace=True)

df_agg = (
    df_sales.groupby(['date_block_num', 'item_id'])
    .agg({
        'item_cnt_day': 'sum',
        'item_price': 'median',
        'item_sales': 'sum',
    })
    .reset_index()
    .rename(columns={
        'item_cnt_day': 'item_cnt_month_allshops',
        'item_price': 'median_price_allshops',
        'item_sales': 'item_sales_month_allshops',
    })
)
df_train = df_train.merge(df_agg, how='left', on=['date_block_num', 'item_id'])
df_train.median_price.fillna(df_train.median_price_allshops, inplace=True)

for cols in [['shop_id'], ['item_id'], ['item_category_id'], ['shop_id', 'item_id'], ['shop_id', 'item_category_id']]:
    df_agg = (
        df_train.groupby(['date_block_num'] + cols)
        .item_cnt_month
        .mean()
        .reset_index()
        .rename(columns={'item_cnt_month': 'targetenc_' + ('|'.join(cols))})
    )
    df_train = df_train.merge(df_agg, how='left', on=['date_block_num']+cols)
    
for col in ['shop_id', 'item_id', 'item_category_id']:
    df_agg = (
        df_train.groupby(col)
        .date_block_num
        .min()
        .reset_index()
        .rename(columns={'date_block_num': col + '_age'})
    )
    df_train = df_train.merge(df_agg, how='left', on=[col])
    df_train[col + '_age'] = df_train.date_block_num - df_train[col + '_age']
    

df_train.head().T

,0,1,2,3,4
date_block_num,0.000000,0.000000,0.000000,0.000000,0.000000
shop_id,0.000000,0.000000,0.000000,0.000000,0.000000
item_id,19.000000,27.000000,28.000000,29.000000,32.000000
item_category_id,40.000000,19.000000,30.000000,23.000000,40.000000
item_sales_month,0.000000,0.000000,0.000000,0.000000,1326.000000
item_cnt_month,0.000000,0.000000,0.000000,0.000000,6.000000
median_price,28.000000,2499.000000,549.000000,2499.000000,221.000000
item_sales_month_allshops,28.000000,16275.000000,4392.000000,9387.000000,101515.860000
item_cnt_month_allshops,1.000000,7.000000,8.000000,4.000000,299.000000
median_price_allshops,28.000000,2499.000000,549.000000,2499.000000,349.000000


In [6]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

#df_train['price_delta'] = df_train.median_price - df_train.median_price_allshops
#df_train['price_ratio'] = df_train.median_price / df_train.median_price_allshops
df_train['item_cnt_ratio'] = sigmoid(df_train.item_cnt_month / df_train.item_cnt_month_allshops)
df_train['item_cnt_ratio'].fillna(0.0, inplace=True)
#df_train['item_sales_ratio'] = df_train.item_sales_month / df_train.item_sales_month_allshops
#df_train['item_sales_ratio'].fillna(0.0, inplace=True)
df_train.head().T

,0,1,2,3,4
date_block_num,0.000000,0.000000,0.000000,0.000000,0.000000
shop_id,0.000000,0.000000,0.000000,0.000000,0.000000
item_id,19.000000,27.000000,28.000000,29.000000,32.000000
item_category_id,40.000000,19.000000,30.000000,23.000000,40.000000
item_sales_month,0.000000,0.000000,0.000000,0.000000,1326.000000
item_cnt_month,0.000000,0.000000,0.000000,0.000000,6.000000
median_price,28.000000,2499.000000,549.000000,2499.000000,221.000000
item_sales_month_allshops,28.000000,16275.000000,4392.000000,9387.000000,101515.860000
item_cnt_month_allshops,1.000000,7.000000,8.000000,4.000000,299.000000
median_price_allshops,28.000000,2499.000000,549.000000,2499.000000,349.000000


In [7]:
df_train.isnull().sum()

date_block_num                        0
shop_id                               0
item_id                               0
item_category_id                      0
item_sales_month                      0
item_cnt_month                        0
median_price                          0
item_sales_month_allshops             0
item_cnt_month_allshops               0
median_price_allshops                 0
targetenc_shop_id                     0
targetenc_item_id                     0
targetenc_item_category_id            0
targetenc_shop_id|item_id             0
targetenc_shop_id|item_category_id    0
shop_id_age                           0
item_id_age                           0
item_category_id_age                  0
item_cnt_ratio                        0
dtype: int64

In [8]:
from functools import partial

def rename_column(index, col_name):
    if col_name in ['shop_id', 'item_id']:
        return col_name
    return '{}_{}'.format(col_name, index)
    
def make_features(label_block):
    label_df = df_train.loc[df_train.date_block_num == label_block, ['shop_id', 'item_id', 'item_cnt_month']].reset_index(drop=True)
    y = label_df.item_cnt_month#.map(lambda x: max(0, min(40, x)))
    return make_features2(label_block, label_df), y

def make_features2(label_block, label_df):
    X = label_df[['shop_id', 'item_id']]
    for i in [1,2,12,13]:
        feature_block = label_block - i
        dff = (
            df_train.loc[df_train.date_block_num == feature_block]
            .drop(['date_block_num', 'item_category_id'], axis=1)
            .rename(partial(rename_column, i), axis='columns')
        )
        X = X.merge(dff, how='left', on=['shop_id', 'item_id'])
    X['item_cnt_month_delta1'] = X['item_cnt_month_1'] - X['item_cnt_month_2']
    X['item_cnt_ratio_delta1'] = X['item_cnt_ratio_1'] - X['item_cnt_ratio_2']
    X['item_cnt_month_allshops_delta1'] = X['item_cnt_month_allshops_1'] - X['item_cnt_month_allshops_2']
    X['item_cnt_month_delta12'] = X['item_cnt_month_12'] - X['item_cnt_month_13']
    X['item_cnt_ratio_delta12'] = X['item_cnt_ratio_12'] - X['item_cnt_ratio_13']
    X['item_cnt_month_allshops_delta12'] = X['item_cnt_month_allshops_12'] - X['item_cnt_month_allshops_13']
    columns_to_drop = (
        ['shop_id', 'item_id']
        + [col for col in X.columns
           if any([col.endswith('_{}'.format(i)) for i in [2,12,13]])])
    X.drop(columns_to_drop, axis=1, inplace=True)
    X.fillna(0, inplace=True)
    scaler = StandardScaler(copy=False)
    scaler.fit_transform(X.values)
    return X

In [18]:
# <34: VALIDATION
# =34: TEST
PREDICTION_BLOCK = 34
TRAINING_BLOCK_RANGE = range(10, PREDICTION_BLOCK)

Xs = []
ys = []
for label_block in TRAINING_BLOCK_RANGE:
    X, y = make_features(label_block)
    Xs.append(X)
    ys.append(y)

X_train = pd.concat(Xs)
y_train = pd.concat(ys)
X_train.head(10).T

,0,1,2,3,4,5,6,7,8,9
item_sales_month_1,-0.072783,-0.072783,-0.072783,-0.072783,-0.072783,-0.072783,-0.072783,-0.072783,-0.072783,-0.072783
item_cnt_month_1,-0.092390,-0.092390,-0.092390,-0.092390,-0.092390,-0.092390,-0.092390,-0.092390,-0.092390,-0.092390
median_price_1,0.375166,-0.334594,-0.304436,0.030643,-0.324744,-0.273975,-0.324744,-0.324744,-0.324744,-0.223205
item_sales_month_allshops_1,-0.053178,-0.083277,-0.058425,0.004569,-0.017440,-0.047271,-0.092155,-0.087817,-0.085103,-0.085263
item_cnt_month_allshops_1,-0.083450,-0.047878,0.076625,0.085518,0.378987,0.156662,-0.119022,-0.083450,-0.056771,-0.092343
median_price_allshops_1,0.374910,-0.335764,-0.305568,0.029943,-0.325902,-0.275067,-0.325902,-0.325902,-0.325902,-0.224232
targetenc_shop_id_1,-0.663279,-0.663279,-0.663279,-0.663279,-0.663279,-0.663279,-0.663279,-0.663279,-0.663279,-0.663279
targetenc_item_id_1,-0.083450,-0.047878,0.076625,0.085518,0.378987,0.156662,-0.119022,-0.083450,-0.056771,-0.092343
targetenc_item_category_id_1,0.478737,0.667903,-0.019786,-0.063253,-0.019786,-0.063253,-0.019786,-0.063253,-0.019786,-0.094049
targetenc_shop_id|item_id_1,-0.092390,-0.092390,-0.092390,-0.092390,-0.092390,-0.092390,-0.092390,-0.092390,-0.092390,-0.092390


In [ ]:
params = {'colsample_bytree': 0.8222460001474794,
 'learning_rate': 0.18120176621156048,
 'max_depth': 8,
 'min_data_in_leaf': 10,
 'num_leaves': 120,
 'reg_lambda': 6.799588683505174,
 'subsample': 0.7903697367193263}

reg = LGBMRegressor(
    n_jobs=4,
    random_state=42,
    n_estimators=300,
    **params,
)

if PREDICTION_BLOCK < 34:
    X_val, y_val = make_features(VALIDATION_BLOCK)
    reg.fit(X_train, y_train, early_stopping_rounds=200, eval_set=[(X_val, y_val)], eval_metric='mse', verbose=True)
else:
    reg.fit(X_train, y_train, eval_metric='mse', verbose=True)

In [ ]:
def clip(s):
    return s.map(lambda x: max(0.0, min(20.0, x)))

def rmse(x, y):
    return math.sqrt(mean_squared_error(clip(x), clip(y)))

if PREDICTION_BLOCK < 34:
    y_pred = pd.Series(reg.predict(X_val))
    print('validation RMSE score:', rmse(y_val, y_pred))

In [ ]:
y_train_p = pd.Series(reg.predict(X_train))
print('training RMSE score', rmse(y_train, y_train_p))

In [ ]:
feature_importances = pd.DataFrame({'feature': X.columns, 'importance': reg.feature_importances_})
feature_importances.sort_values(by='importance', ascending=True, inplace=True)
feature_importances.plot(x='feature', y='importance', kind='barh', sort_columns=True, figsize=(10,10))

In [ ]:
import hyperopt

def objective(params):
    params['num_leaves'] = int(params['num_leaves'])
    params['max_depth'] = int(params['max_depth'])
    params['min_data_in_leaf'] = int(params['min_data_in_leaf'])
    
    model = LGBMRegressor(random_state=42, n_jobs=-1, **params)
    
    model.fit(X_train, y_train, early_stopping_rounds=200, eval_set=[(X_val, y_val)], verbose=False)
    y_pred = pd.Series(model.predict(X_val))
    score = rmse(y_val, y_pred)
    print("RMSE {:.6f} params {}".format(score, params))
    return -score

space = {
    'num_leaves': hyperopt.hp.quniform('num_leaves', 50, 200, 10),
    'max_depth': hyperopt.hp.quniform('max_depth', 3, 10, 1),
    'min_data_in_leaf': hyperopt.hp.quniform('min_data_in_leaf',  5, 25, 2),
    'colsample_bytree': hyperopt.hp.uniform('colsample_bytree', 0.5, 1.0),
    'learning_rate': hyperopt.hp.uniform('learning_rate', 0.03, 0.2),
    'subsample': hyperopt.hp.uniform('subsample', 0.5, 1.0),
    'reg_lambda': hyperopt.hp.uniform('reg_lambda', 0.1, 10.0),
}

if PREDICTION_BLOCK < 34:
    best_params = hyperopt.fmin(
        fn=objective,
        space=space,
        algo=hyperopt.tpe.suggest,
        max_evals=100)
    best_params

In [ ]:
if PREDICTION_BLOCK == 34:
    X_test = make_features2(PREDICTION_BLOCK, df_test)
    y_test_pred = clip(pd.Series(reg.predict(X_test)))
    pd.DataFrame(y_test_pred).info()
    y_test_pred.head()

In [ ]:
    df_submit = pd.DataFrame(df_test.loc[:,'ID'])
    df_submit['item_cnt_month'] = y_test_pred
    df_submit.to_csv('lgbm_tuned1.csv', index=False)